In [3]:
import pandas as pd 
import numpy as np 
from datetime import timedelta 
import warnings 
warnings.filterwarnings('ignore') 

def load_and_clean_data(irop_file, passenger_file): 

    try:
        irop_df = pd.read_csv(irop_file, encoding='utf-8-sig')
    except UnicodeDecodeError:
        try:
            irop_df = pd.read_csv(irop_file, encoding='cp1252')
        except UnicodeDecodeError:
            irop_df = pd.read_csv(irop_file, encoding='latin1')

    try:
        pax_df = pd.read_csv(passenger_file, encoding='utf-8-sig')
    except UnicodeDecodeError:
        try:
            pax_df = pd.read_csv(passenger_file, encoding='cp1252')
        except UnicodeDecodeError:
            pax_df = pd.read_csv(passenger_file, encoding='latin1')

    irop_df.columns = irop_df.columns.str.strip() 
    pax_df.columns = pax_df.columns.str.strip() 
    irop_df['Flight_Date'] = pd.to_datetime(irop_df['Flight_Date'], errors='coerce') 
    pax_df['Flight_Date'] = pd.to_datetime(pax_df['Flight_Date'], errors='coerce') 

    if 'Delay_Bucket' in irop_df.columns:
        print("Original delay buckets:", sorted(irop_df['Delay_Bucket'].dropna().unique()))

        replacements = {
            'â€"': '–',        
            'â€"': '–',        
            'â€œ': '', 'â€': '', 'â€™': "'", 'â€˜': "'", 
            'Â': '', 'â': ''   
        }

        for old, new in replacements.items():
            irop_df['Delay_Bucket'] = irop_df['Delay_Bucket'].str.replace(old, new, regex=False)

        irop_df['Delay_Bucket'] = irop_df['Delay_Bucket'].str.replace(r'15[^\d]*60\s*MINS', '15–60 MINS', regex=True)
        irop_df['Delay_Bucket'] = irop_df['Delay_Bucket'].str.replace(r'1[^\d]*3\s*HOURS', '1–3 HOURS', regex=True)
        irop_df['Delay_Bucket'] = irop_df['Delay_Bucket'].str.replace(r'3[^\d]*8\s*HOURS', '3–8 HOURS', regex=True)
        irop_df['Delay_Bucket'] = irop_df['Delay_Bucket'].str.replace(r'8[^\d]*\+\s*HOURS', '8+ HOURS', regex=True)

        print("Cleaned delay buckets:", sorted(irop_df['Delay_Bucket'].dropna().unique()))

    return irop_df, pax_df

def map_controllability(irop_df): 
    controllable = [ 
        'DELAY - Carrier Caused', 'DELAY - Late Arriving Aircraft', 
        'CANCELLATION - Carrier Caused', 'DIVERSION - Carrier Caused', 
        'DIVERSION - Late Arriving Aircraft' 
    ] 
    uncontrollable = [ 
        'DELAY - Weather', 'DELAY - National Aviation System', 'DELAY - Security', 
        'CANCELLATION - Weather', 'CANCELLATION - National Aviation System', 
        'DIVERSION - Weather', 'DIVERSION - National Aviation System' 
    ] 
    irop_df['Controllability_Mapped'] = irop_df['IROP_Type'].apply( 
        lambda x: 'CONTROLLABLE' if x in controllable else 'UNCONTROLLABLE' if x in uncontrollable else 'UNKNOWN' 
    ) 
    return irop_df 

def filter_disruptions(irop_df): 

    target_patterns = ['15–60 MINS', '1–3 HOURS', '3–8 HOURS', '8+ HOURS']

    print(f"Target delay patterns: {target_patterns}")
    print(f"Actual delay buckets in data: {sorted(irop_df['Delay_Bucket'].dropna().unique())}")

    matched_buckets = irop_df['Delay_Bucket'].isin(target_patterns)
    print(f"Rows matching delay patterns: {matched_buckets.sum()}")

    cancellation_diversion = irop_df['IROP_Type'].str.contains('CANCELLATION|DIVERSION', na=False)
    print(f"Rows with cancellation/diversion: {cancellation_diversion.sum()}")

    total_filtered = cancellation_diversion | matched_buckets
    print(f"Total rows after filtering: {total_filtered.sum()}")

    return irop_df[total_filtered].copy() 

def filter_last_30_days(df, column='Flight_Date'): 
    reference = df[column].max() 
    return df[df[column] >= (reference - timedelta(days=30))] 

def get_high_value_customers(pax_df, threshold=1895): 
    return pax_df[ 
        (pax_df['Revenue_Last_12mo'] >= threshold) & 
        (pax_df['Customer_Value_Tier'] == 'HIGH_VALUE') 
    ].copy() 

def merge_datasets(irop_df, pax_df): 
    """
    Enhanced merge function to handle Flight_Key matching with hierarchical fallback
    """
    merged_list = []

    merged_1 = pd.merge(irop_df, pax_df, left_on='Flight_Key', right_on='Flight_Key_1', how='inner')
    if not merged_1.empty:
        merged_1['Used_Flight_Key'] = 'Flight_Key_1'
        merged_1['Flight_Sequence'] = 1
        merged_list.append(merged_1)

    merged_2 = pd.merge(irop_df, pax_df, left_on='Flight_Key', right_on='Flight_Key_2', how='inner')
    if not merged_2.empty:
        merged_2['Used_Flight_Key'] = 'Flight_Key_2'
        merged_2['Flight_Sequence'] = 2
        merged_list.append(merged_2)

    merged_3 = pd.merge(irop_df, pax_df, left_on='Flight_Key', right_on='Flight_Key_3', how='inner')
    if not merged_3.empty:
        merged_3['Used_Flight_Key'] = 'Flight_Key_3'
        merged_3['Flight_Sequence'] = 3
        merged_list.append(merged_3)

    if merged_list:
        merged = pd.concat(merged_list, ignore_index=True)

        merged = merged.drop_duplicates(subset=['Unique_Customer_Id', 'Flight_Key'])
        return merged

    print("Flight_Key matching failed, trying Flight_ID fallback...")
    merged = pd.merge(irop_df, pax_df, left_on='Flight_ID', right_on='Flightkey1', how='inner')
    if merged.empty: 
        pax_long = [] 
        for _, row in pax_df.iterrows(): 
            for fk in ['Flightkey1', 'Flightkey2', 'Flightkey3']: 
                if pd.notna(row.get(fk)): 
                    temp = row.copy() 
                    temp['Match_Key'] = row[fk] 
                    pax_long.append(temp) 
        pax_long_df = pd.DataFrame(pax_long) 
        merged = pd.merge(irop_df, pax_long_df, left_on='Flight_ID', right_on='Match_Key', how='inner') 
        if not merged.empty:
            merged['Used_Flight_Key'] = 'Fallback_Match'
            merged['Flight_Sequence'] = 0

    return merged 

def analyze_disruption_timing(detail_df):
    """Analyze timing between disrupted legs for intervention opportunities"""

    print(f"Available columns: {list(detail_df.columns)}")

    date_columns = [col for col in detail_df.columns if 'flight_date' in col.lower() or 'date' in col.lower()]
    print(f"Date columns found: {date_columns}")

    if not date_columns:
        print("WARNING: No flight date column found - skipping timing analysis")
        detail_df['Intervention_Opportunity'] = False
        return detail_df

    flight_date_col = date_columns[0]
    print(f"Using date column: {flight_date_col}")

    detail_df[flight_date_col] = pd.to_datetime(detail_df[flight_date_col], errors='coerce')

    detail_df = detail_df.sort_values(['Unique_Customer_Id', 'Flight_Sequence', flight_date_col])

    detail_df['Next_Flight_Date'] = detail_df.groupby('Unique_Customer_Id')[flight_date_col].shift(-1)
    detail_df['Hours_To_Next_Flight'] = (
        detail_df['Next_Flight_Date'] - detail_df[flight_date_col]
    ).dt.total_seconds() / 3600

    detail_df['Intervention_Opportunity'] = (
        (detail_df['Flight_Sequence'] == 1) & 
        (detail_df['Hours_To_Next_Flight'].between(2, 48)) &  
        (detail_df['Controllability_Mapped'] == 'CONTROLLABLE')
    )

    return detail_df

def validate_data_quality(detail_df):
    """Flag potential data quality issues"""

    if 'Unique_Customer_Id' not in detail_df.columns:
        print("WARNING: Missing Unique_Customer_Id column - skipping data quality validation")
        detail_df['Suspicious_High_Count'] = False
        detail_df['Potential_Travel_Agent'] = False
        return detail_df

    flight_id_col = 'Flight_ID' if 'Flight_ID' in detail_df.columns else 'Flight_Key'
    high_disruption = detail_df.groupby('Unique_Customer_Id')[flight_id_col].nunique()
    suspicious_customers = high_disruption[high_disruption > 10].index.tolist()

    if suspicious_customers:
        print(f"WARNING: {len(suspicious_customers)} customers with 10+ disrupted flights")
        detail_df['Suspicious_High_Count'] = detail_df['Unique_Customer_Id'].isin(suspicious_customers)
    else:
        detail_df['Suspicious_High_Count'] = False

    if 'Email_Address' in detail_df.columns:
        detail_df['Potential_Travel_Agent'] = detail_df['Email_Address'].str.contains(
            r'travel|agent|corporate|group', case=False, na=False
        )

        if detail_df['Potential_Travel_Agent'].any():
            print(f"{detail_df['Potential_Travel_Agent'].sum()} records match travel agent email patterns")
    else:
        print("WARNING: No Email_Address column found - skipping travel agent detection")
        detail_df['Potential_Travel_Agent'] = False

    return detail_df

def calculate_matrix_based_comp(row): 
    disruption = row['IROP_Type'] 
    controllability = row['Controllability_Mapped'] 
    delay_minutes = row.get('Delay_Minutes', 0) 
    delay_minutes = 0 if pd.isna(delay_minutes) else delay_minutes 

    if controllability != 'CONTROLLABLE':
        return 0

    if 'CANCELLATION' in disruption: 
        return 100 
    if 'DELAY' in disruption: 
        if delay_minutes >= 480: 
            return 150 
        elif delay_minutes >= 180: 
            return 100 
        elif delay_minutes >= 60: 
            return 50 
    if 'DIVERSION' in disruption: 
        return 200 
    return 0 

def calculate_hierarchical_compensation(customer_group):
    """Calculate compensation using hierarchical severity logic for customers with multiple disruptions"""

    severity_order = {
        'CANCELLATION': 4,
        'DIVERSION': 3, 
        'DELAY_8H+': 2,
        'DELAY_3-8H': 2,
        'DELAY_1-3H': 1,
        'DELAY_15-60M': 0
    }

    customer_group['Severity_Score'] = customer_group.apply(
        lambda row: severity_order.get(
            'CANCELLATION' if 'CANCELLATION' in row['IROP_Type']
            else 'DIVERSION' if 'DIVERSION' in row['IROP_Type']
            else f"DELAY_{row['Delay_Bucket']}" if pd.notna(row.get('Delay_Bucket'))
            else 'UNKNOWN', 0
        ), axis=1
    )

    controllable_events = customer_group[customer_group['Controllability_Mapped'] == 'CONTROLLABLE']

    if len(controllable_events) > 0:
        primary_event = controllable_events.loc[controllable_events['Severity_Score'].idxmax()]
        return primary_event['Recommended_Compensation_Amount']

    return 0

def campaign_category(controllability): 
    return { 
        'CONTROLLABLE': 'CONTROLLABLE_ONLY', 
        'UNCONTROLLABLE': 'UNCONTROLLABLE_ONLY' 
    }.get(controllability, 'UNKNOWN') 

def clean_output_columns(detail_df):
    """Clean up redundant columns and rename for clarity"""

    essential_cols = [

        'Flight_ID', 'Flight_Key', 'Flight_Number', 'Origin_Airport', 'Destination_Airport',
        'IROP_Type', 'Delay_Bucket', 'Delay_Minutes', 'Controllability_Mapped',

        'Unique_Customer_Id', 'Guest_Name', 'Email_Address', 'HomeAirport',
        'Revenue_Last_12mo', 'Customer_Value_Tier', 'Activity_Level',

        'Itinerary', 'TripType', 'Net_Revenue', 'BOOKING_CHANNEL',

        'Flight_Sequence', 'Used_Flight_Key', 'Recommended_Compensation_Amount', 
        'Campaign_Category', 'Intervention_Opportunity', 'Suspicious_High_Count', 
        'Potential_Travel_Agent', 'Hours_To_Next_Flight'
    ]

    if 'Flight_Date_x' in detail_df.columns:
        detail_df['Flight_Date'] = detail_df['Flight_Date_x']
        essential_cols.append('Flight_Date')
    elif 'Flight_Date_y' in detail_df.columns:
        detail_df['Flight_Date'] = detail_df['Flight_Date_y']
        essential_cols.append('Flight_Date')

    available_cols = [col for col in essential_cols if col in detail_df.columns]

    if 'Controllability' in detail_df.columns and 'Controllability_Mapped' in detail_df.columns:
        available_cols = [col for col in available_cols if col != 'Controllability']

    print(f"Cleaning columns: keeping {len(available_cols)} of {len(detail_df.columns)} total columns")

    return detail_df[available_cols].copy()

def summarize_and_export(detail_df): 

    detail_df = clean_output_columns(detail_df)

    detail_df['Recommended_Compensation_Amount'] = detail_df.apply(calculate_matrix_based_comp, axis=1) 
    detail_df['Campaign_Category'] = detail_df['Controllability_Mapped'].apply(campaign_category) 

    campaign_agg = {
        "Unique_Customer_Id": "nunique",
        "Net_Revenue": "sum", 
        "Recommended_Compensation_Amount": "sum"
    }

    flight_col_for_campaign = 'Flight_ID' if 'Flight_ID' in detail_df.columns else 'Flight_Key'
    campaign_agg[flight_col_for_campaign] = "nunique"

    campaign_summary = ( 
        detail_df.groupby("Campaign_Category") 
                 .agg(campaign_agg) 
                 .reset_index() 
                 .rename(columns={ 
                    "Unique_Customer_Id": "Customers", 
                    flight_col_for_campaign: "Disrupted_Flights", 
                    "Net_Revenue": "Net_Revenue_Impact", 
                    "Recommended_Compensation_Amount": "Total_Comp_Budget" 
                 }) 
    ) 

    print("\n=== FLIGHT-LEVEL CAMPAIGN SUMMARY ===\n") 
    for _, row in campaign_summary.iterrows(): 
        print(f"- {row['Campaign_Category']}:") 
        print(f"   Customers: {int(row['Customers']):,}") 
        print(f"   Disrupted Flights: {int(row['Disrupted_Flights']):,}") 
        print(f"   Net Revenue Impact: ${row['Net_Revenue_Impact']:,.0f}") 
        print(f"   Total Compensation Budget: ${row['Total_Comp_Budget']:,.0f}\n") 

    total_cust = campaign_summary['Customers'].sum() 
    total_budget = campaign_summary['Total_Comp_Budget'].sum() 
    print(f"TOTAL — {total_cust:,} customers, ${total_budget:,.0f} total comp budget")

    detail_df.to_csv("high_value_disrupted_flights_detailed.csv", index=False, encoding='utf-8-sig')

    print("\nFile written: high_value_disrupted_flights_detailed.csv")

def main(): 
    print("Running enhanced flight-level disruption analysis...") 
    irop_df, pax_df = load_and_clean_data("irrop2.5_july.csv", "pass2.5_july.csv") 
    irop_df = map_controllability(irop_df) 
    irop_df = filter_disruptions(irop_df) 
    irop_df = filter_last_30_days(irop_df) 
    pax_df = filter_last_30_days(pax_df) 
    high_value_pax = get_high_value_customers(pax_df) 

    print(f"High value customers: {len(high_value_pax):,}")
    print(f"IROP events: {len(irop_df):,}")

    merged = merge_datasets(irop_df, high_value_pax) 

    if len(merged) == 0:
        print("No matches found - check Flight_Key mapping")
        return

    print(f"Matched disruptions: {len(merged):,}")
    print(f"Unique customers with disruptions: {merged['Unique_Customer_Id'].nunique():,}")

    print(f"\nMerged dataframe columns: {list(merged.columns)}")
    print(f"Merged dataframe shape: {merged.shape}")

    merged = analyze_disruption_timing(merged)
    merged = validate_data_quality(merged)

    summarize_and_export(merged) 

    intervention_opps = merged[merged.get('Intervention_Opportunity', False)]
    if len(intervention_opps) > 0:
        print(f"\nINTERVENTION OPPORTUNITIES: {len(intervention_opps)} first-leg disruptions with upcoming flights")
        print("These customers experienced controllable delays on their first flight with 2-48 hours until next flight")

if __name__ == "__main__": 
    main()

Running enhanced flight-level disruption analysis...
Original delay buckets: ['15–60 MINS', '1–3 HOURS', '3–8 HOURS', '8+ HOURS']
Cleaned delay buckets: ['15–60 MINS', '1–3 HOURS', '3–8 HOURS', '8+ HOURS']
Target delay patterns: ['15–60 MINS', '1–3 HOURS', '3–8 HOURS', '8+ HOURS']
Actual delay buckets in data: ['15–60 MINS', '1–3 HOURS', '3–8 HOURS', '8+ HOURS']
Rows matching delay patterns: 4064
Rows with cancellation/diversion: 103
Total rows after filtering: 4167
High value customers: 52,493
IROP events: 4,167
Matched disruptions: 12,314
Unique customers with disruptions: 10,938

Merged dataframe columns: ['Flight_ID', 'Flight_Key', 'Flight_Number', 'Flight_Date_x', 'Origin_Airport', 'Destination_Airport', 'IROP_Type', 'Delay_Bucket', 'Controllability', 'Delay_Minutes', 'Controllability_Mapped', 'Unique_Customer_Id', 'Itinerary', 'BOOKING_DATETIME', 'TripType', 'Physical_Pax', 'Gross_Revenue', 'Net_Revenue', 'BOOKING_CHANNEL', 'Travel_Protection', 'Bundle_Flag', 'Email_Address', 'Gu